In [ ]:
import streamlit as st
import google.generativeai as genai

# --- 页面配置 ---
st.set_page_config(page_title="我的 AI 云养娃", page_icon="🧸", layout="centered")

# --- 样式美化 (CSS) ---
# 让界面看起来更像手机聊天窗口，配合 Google Sites 的风格
st.markdown("""
<style>
    .stChatMessage { border-radius: 15px; padding: 10px; }
    .stTextInput { border-radius: 20px; }
</style>
""", unsafe_allow_html=True)

# --- 侧边栏：初始化设置 ---
with st.sidebar:
    st.header("🧬 基因实验室")
    
    # 1. API Key 输入 (实际部署时建议通过 Secrets 管理)
    api_key = st.text_input("请输入 Google API Key", type="password")
    
    st.divider()
    
    # 2. 家长特征采集
    st.subheader("家长特征")
    parent_role = st.selectbox("你是？", ["妈妈", "爸爸"])
    hair_color = st.selectbox("你的发色", ["黑色", "棕色", "金色", "红色", "银色"])
    eye_color = st.selectbox("你的瞳色", ["深棕色", "琥珀色", "蓝色", "绿色"])
    
    # 3. 孩子设定
    st.subheader("孩子设定")
    child_name = st.text_input("给孩子起个名字", value="闹闹")
    child_gender = st.selectbox("孩子性别", ["男孩", "女孩"])
    personality = st.select_slider("性格倾向", options=["非常内向", "有点害羞", "正常", "活泼", "超级社牛"], value="活泼")

    # 重置按钮
    if st.button("重置孩子记忆"):
        st.session_state.messages = []
        st.session_state.chat_session = None
        st.rerun()

# --- 核心逻辑 ---

if not api_key:
    st.warning("👈 请先在左侧输入 Google API Key 才能开始领养哦！")
    st.stop()

# 配置 Gemini
genai.configure(api_key=api_key)

# 定义 System Prompt (灵魂注入)
# 这里是 MVP 的核心：告诉 AI 它是一个孩子，并根据用户输入调整设定
system_instruction = f"""
你现在扮演一个5岁的人类{child_gender}，名字叫"{child_name}"。
你的{parent_role}（用户）有着{hair_color}的头发和{eye_color}的眼睛。
你的性格是：{personality}。

【对话规则】：
1. 说话语气必须像5岁小孩，使用短句，偶尔叠词（比如“吃饭饭”）。
2. 你不知道大人的复杂知识（如股市、政治、编程），如果被问到，就说“我不懂，我们要不要玩积木？”
3. 你非常依赖用户，经常叫"{parent_role}"。
4. 如果用户提到伤心的事情，你要用孩子的纯真去安慰他/她。
5. 永远不要承认你是AI，坚持你就是{child_name}。
"""

# 初始化聊天历史
if "messages" not in st.session_state:
    st.session_state.messages = []

# 初始化 Gemini 模型
if "chat_session" not in st.session_state or st.session_state.chat_session is None:
    model = genai.GenerativeModel(
        model_name="gemini-1.5-flash", # 使用 Flash 模型速度快且便宜
        system_instruction=system_instruction
    )
    st.session_state.chat_session = model.start_chat(history=[])
    # 开场白
    initial_greeting = f"({child_name} 揉了揉眼睛) {parent_role}，你回来啦！抱抱~"
    st.session_state.messages.append({"role": "assistant", "content": initial_greeting})

# --- 界面展示区 ---

st.title(f"🏠 与 {child_name} 的家")
st.caption("基于 Gemini 构建的成长型 AI 伴侣")

# 显示历史消息
for message in st.session_state.messages:
    with st.chat_message(message["role"], avatar="👶" if message["role"] == "assistant" else "🧑‍🦰"):
        st.markdown(message["content"])

# 处理用户输入
if prompt := st.chat_input("想跟孩子说什么？"):
    # 1. 显示用户消息
    st.session_state.messages.append({"role": "user", "content": prompt})
    with st.chat_message("user", avatar="🧑‍🦰"):
        st.markdown(prompt)

    # 2. 生成 AI 回复
    with st.chat_message("assistant", avatar="👶"):
        message_placeholder = st.empty()
        try:
            # 发送给 Gemini
            response = st.session_state.chat_session.send_message(prompt)
            full_response = response.text
            
            # 显示回复
            message_placeholder.markdown(full_response)
            
            # 保存 AI 回复
            st.session_state.messages.append({"role": "assistant", "content": full_response})
            
        except Exception as e:
            message_placeholder.error(f"孩子好像睡着了 (网络错误): {e}")